# SA405 Fall 2021
## MiniProject 2: VRP
#### Vehicle Routing Problem with subtour elimination constraint generation.

Instructions:
1. Complete the steps below to create your base VRP model
2. As the "user", add subtour elimination constraints until an optimal solution to the VRP problem is obtained (remember that the "hub" is never a part of the subtour elimination constraints)
3. Submit a your code with output displayed
4. Recall that, as an optional bonus opportunity, you can try to automate the subtour elimination process.

In [1]:
import pyomo.environ as pyo
import math

### Step 1: Define your sets and parameters.

In [2]:
#Hint:  Print each data element to make sure it is what you need it to be

# Sets, all nodes, customers and edges
NODES =  list(range(17))  # [0,1,...,16]
CUSTOMERS = [v for v in NODES if v>0]
EDGES = [(v,w) for v in NODES for w in NODES if v < w]


# dictionary of vertex coordinates
# (key:value = vertex:(x-coord,y-coord))
# location 15 is at (0,0)
COORD = {0:(4,4),1:(2,8),2:(8,8),3:(0,7),
         4:(1,7),5:(5,6),6:(7,6),7:(4,5),
         8:(6,5),9:(5,3),10:(8,3),11:(1,2),
         12:(2,2),13:(3,1),14:(7,1),15:(0,0),
         16:(0,8)}
#Parameters
         
#Distance between every two nodes. There are multiple ways to do this but you must
# use the math.sqrt() function to compute the linear distance

DISTANCE = {(v,w):math.sqrt((COORD[v][0]-COORD[w][0])**2 +(COORD[v][1]-COORD[w][1])**2)
            for v in NODES for w in NODES if v < w}
         
# Also have a max capacity and number of vehicles to define
NUM_TRUCKS = 4
CAPACITY = 5

### Step 2: Build your base VRP model with no subtour elimination constraints like we did in class.
### Recall that there were only 2 constraints in our model

#### Decision Variables

In [3]:
# Create model, define binary variables
model = pyo.ConcreteModel()

model.x = pyo.Var(EDGES,domain=pyo.Binary)

#### Objective function

In [4]:
# Minimize total distance traveled
def obj_rule(model):
    return sum(DISTANCE[v,w]*model.x[v,w] for v,w in EDGES)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

#### Constraints

In [5]:
# 8 edges to node 0
def hub_degree_rule(model):
    return sum(model.x[0,c] for c in CUSTOMERS)==2*NUM_TRUCKS
model.hub_degree_const = pyo.Constraint(rule=hub_degree_rule)

# 2 edges to every customer
def cust_degree_rule(model, cust):
    return sum(model.x[v,w] for v,w in EDGES if v==cust or w==cust)==2
model.cust_degree_rule = pyo.Constraint(CUSTOMERS,rule=cust_degree_rule)

### Step 3: Solve the problem and print your solution.

In [6]:
solver_result = pyo.SolverFactory('glpk').solve(model)
solve_status = solver_result.solver.termination_condition
print(solve_status)

optimal


In [7]:
solution=[(i,j) for i,j in EDGES if model.x[i,j]==1]
print(f'Cumulative distance: {model.obj()}')
print(f'Edges: {solution}\n')

Cumulative distance: 44.59856358193051
Edges: [(0, 2), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (0, 12), (0, 13), (1, 4), (1, 16), (2, 6), (3, 4), (3, 16), (5, 7), (6, 8), (9, 14), (10, 14), (11, 12), (11, 15), (13, 15)]



### Step 4: Now, you need to iteratively add subtour elimination constraints to your model, resolve and get an optimal solution. 

You can do this manually like I did in the lesson 10 supplement file, or you can do it
in an automated fashion. If you can figure out how to automate this process correctly with 
no user input then I will give you bonus points on your grade.

Remember that each time you add a new constraint, you must re-solve, print your solution, determine if it
is optimal and so on. I will walk you through the first iteration.

#### Step 4.1: Is your current solution optimal, why or why not?

To answer: No, it is not optimal because there is a subtour on nodes 1, 4, 3, and 16

#### Step 4.2: Define a set S1 to eliminate the subtour you want to eliminate. Also, define a function to create a constraint which eliminates this subtour and add this constraint to your model.

In [8]:
# Set on which I want to eliminate a subtour
S1 = [1,4,3,16]
# Num cars needed for this tour
C_s = math.ceil(len(S1)/CAPACITY)
def subtour_elim1(model):
    return sum (model.x[i,j] for i,j in EDGES if i in S1 if j in S1) <= len(S1)-C_s
model.subtour1_constr = pyo.Constraint(rule = subtour_elim1)

#### Step 4.3: Re-solve the problem and print your solution. Is this solution optimal?

In [9]:
solver_result = pyo.SolverFactory('glpk').solve(model)
solve_status = solver_result.solver.termination_condition
print(solve_status)
solution=[(i,j) for i,j in EDGES if model.x[i,j]==1]
print(f'Cumulative distance: {model.obj()}')
print(f'Edges: {solution}\n')

optimal
Cumulative distance: 47.11916678656623
Edges: [(0, 1), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 12), (0, 13), (1, 16), (2, 6), (2, 10), (3, 4), (3, 16), (5, 7), (6, 8), (9, 14), (10, 14), (11, 12), (11, 15), (13, 15)]



#### Continue until you have an optimal solution. Report the optimal solution and the routes that each delivery person will take.

In [10]:
# Not optimal, tour too long 0-8-6-2-10-14-9-0
S2 = [8,6,2,10,14,9]
C_s = math.ceil(len(S2)/CAPACITY)
def subtour_elim2(model):
    return sum (model.x[i,j] for i,j in EDGES if i in S2 if j in S2) <= len(S2)-C_s
model.subtour2_constr = pyo.Constraint(rule = subtour_elim2)

In [11]:
solver_result = pyo.SolverFactory('glpk').solve(model)
solve_status = solver_result.solver.termination_condition
print(solve_status)
solution=[(i,j) for i,j in EDGES if model.x[i,j]==1]
print(f'Cumulative distance: {model.obj()}')
print(f'Edges: {solution}\n')

optimal
Cumulative distance: 47.210734275992586
Edges: [(0, 1), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (0, 12), (0, 13), (1, 7), (2, 5), (2, 6), (3, 4), (3, 16), (4, 16), (6, 8), (9, 14), (10, 14), (11, 12), (11, 15), (13, 15)]



In [12]:
# Not optimal subtour on 3-4-16-3
S3 = [3,4,16]
C_s = math.ceil(len(S3)/CAPACITY)
def subtour_elim3(model):
    return sum (model.x[i,j] for i,j in EDGES if i in S3 if j in S3) <= len(S3)-C_s
model.subtour3_constr = pyo.Constraint(rule = subtour_elim3)

In [13]:
solver_result = pyo.SolverFactory('glpk').solve(model)
solve_status = solver_result.solver.termination_condition
print(solve_status)
solution=[(i,j) for i,j in EDGES if model.x[i,j]==1]
print(f'Cumulative distance: {model.obj()}')
print(f'Edges: {solution}\n')

optimal
Cumulative distance: 47.32263543913479
Edges: [(0, 1), (0, 2), (0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (1, 16), (2, 6), (3, 4), (3, 16), (5, 7), (6, 8), (9, 14), (10, 14), (11, 12), (11, 15), (12, 13), (13, 15)]



In [14]:
# Not optimal, subtour on 11-12-13-15-11
S4 = [11,12,13,15]
C_s = math.ceil(len(S4)/CAPACITY)
def subtour_elim4(model):
    return sum (model.x[i,j] for i,j in EDGES if i in S4 if j in S4) <= len(S4)-C_s
model.subtour4_constr = pyo.Constraint(rule = subtour_elim4)

In [15]:
solver_result = pyo.SolverFactory('glpk').solve(model)
solve_status = solver_result.solver.termination_condition
print(solve_status)
solution=[(i,j) for i,j in EDGES if model.x[i,j]==1]
print(f'Cumulative distance: {model.obj()}')
print(f'Edges: {solution}\n')

optimal
Cumulative distance: 47.5670254457392
Edges: [(0, 4), (0, 5), (0, 7), (0, 8), (0, 9), (0, 10), (0, 12), (0, 13), (1, 7), (1, 16), (2, 5), (2, 6), (3, 4), (3, 16), (6, 8), (9, 14), (10, 14), (11, 12), (11, 15), (13, 15)]



In [16]:
# Optimal Hooray
# Tours are:
# 0-4-3-16-1-7-0
# 0-5-2-6-8-0
# 0-9-14-10-0
# 0-12-11-15-13-0